# Задание

In [1]:
import pandas as pd
data = pd.read_csv('../datasets/kaggle_survey_2022_responses.csv')

D:\Temp\ipykernel_21368\4092154101.py:2: DtypeWarning: Columns (0,15,43,57,73,88,104,118,126,132,170,200,208,215,225,248,255,257,260,270,271,272,277,281,294) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../datasets/kaggle_survey_2022_responses.csv')


## Task 1

Узнай самый часто встречаемый возраст респондентов

In [2]:
data['Q2'].value_counts().idxmax()

'18-21'

## Task 2

Посчитай долю женщин среди русских участников опроса, которые уже не студенты. Результат округли до двух цифр после запятой

In [3]:
rus_no_student = data[(data['Q4'].str.contains('Russia')) & data['Q5'].str.contains('No')].shape
rus_no_student_women = data[data['Q3'].str.contains('Woman') & data['Q4'].str.contains('Russia') & data['Q5'].str.contains('No')].shape
round(rus_no_student_women[0] / rus_no_student[0], 2)

0.24

## Task 3

Найди среднее время заполнения опроса в **минутах**. Результат округли до двух знаков после запятой

In [4]:
data.drop(0, inplace=True)
data['Duration (in seconds)'] = data['Duration (in seconds)'].astype(int)
round(data['Duration (in seconds)'].mean() / 60, 2)

168.17

## Task 4

Выведи топ-3 образовательных курса, которые указали респонденты. P.S Это вопрос Q6

In [5]:
# создаём список колонок начинающихся с 'Q6'
q6_columns = [col for col in data.columns if col.startswith('Q6')]
# объединяем данные всех колононок в одну
courses = data[q6_columns].melt(var_name='all_Q6', value_vars=q6_columns)
# подсчитываем количество каждого уникального значения, выбираем топ 3, преобразуем в словарь
courses_top3_dict = courses['value'].value_counts().iloc[:3].to_dict()
print("Топ-3 образовательных курса:")
for i, (courses, count) in enumerate(courses_top3_dict.items(), 1):
    print(f"{i}. {courses} ({count})")

Топ-3 образовательных курса:
1. Coursera (9699)
2. University Courses (resulting in a university degree) (6780)
3. Kaggle Learn Courses (6628)


## Task 5

Узнай топ-5 языков программирования, которые регулярно используют Data Scientists в своей работе

In [6]:
# выбираем только строки со значением 'Data Scientist' в колонке 'Q23'
ds_data = data[data['Q23'] == 'Data Scientist']
# создаём список колонок начинающихся с 'Q12'
q12_columns = [col for col in ds_data.columns if col.startswith('Q12')]
# объединяем данные всех колононок в одну
programming_languages = ds_data[q12_columns].melt(var_name='all_Q12', value_vars=q12_columns)
# подсчитываем количество каждого уникального значения, выбираем топ 5, преобразуем в словарь
programming_languages_top5_dict = programming_languages['value'].value_counts().iloc[:5].to_dict()
print("Топ-5 языков программирования, которые регулярно используют Data Scientists в своей работе:")
for i, (programming_languages, count) in enumerate(programming_languages_top5_dict.items(), 1):
    print(f"{i}. {programming_languages} ({count})")

Топ-5 языков программирования, которые регулярно используют Data Scientists в своей работе:
1. Python (1808)
2. SQL (1140)
3. R (530)
4. Bash (248)
5. Javascript (187)


## Task 6

Узнай средний доход среди 'Data Scientist' специалистов и средний доход среди 'Machine Learning/ MLops Engineer' \
Т.к. в колонке Q29 указан промежуток, то возьми среднее значение из промежутка. 

Например: 
- '15,000-19,999' - 17500, 
- '125,000-149,999' - 137500. 
- \> $1,000,000 - оставляем 1000000

Средние доход **округли до целого.**

In [7]:
# словарь специальностей
job_titles = {
    'DS': 'Data Scientist',
    'ML': 'Machine Learning/ MLops Engineer',
}
# словарь средних зарплат
avg_salaries = {}
# вычисляем среднюю зарплату для каждой специальности
for job_title in job_titles.values():
    # фильтруем данные по специальности
    ds_data = data[data['Q23'] == job_title]
    # получаем список диапазонов зарплат, удаляя лишние символы, лишние строки
    salary_range = [s.replace('$', '').replace(',', '').replace('>', '') for s in ds_data['Q29'].dropna().values.tolist()]
    # вычисляем среднюю зарплату для специальности используя генератор списка, суммируя все элементы списка (усредняя, если они в промежутке)
    avg_salaries[job_title] = round(sum([int(s.split('-')[0]) if len(s.split('-')) == 1 else (int(s.split('-')[0]) + int(s.split('-')[1])) / 2 for s in salary_range]) / len(salary_range))

for job_title, avg_salary in avg_salaries.items():
    print(f'Средний доход для {job_title}: {avg_salary} $USD')

Средний доход для Data Scientist: 70337 $USD
Средний доход для Machine Learning/ MLops Engineer: 67437 $USD


## Дополнительно

Найди какой-нибудь интересный инсайт из этих данных. Пиши свои интересные инсайты в чат проекта с хэштегом.
#инсайт

Автор самого интересного инсайта получит p2p баллы. Поверь, они тебе понадобятся

In [8]:
# выбираем данные для 10 лет и более
data_10 = data[(data['Q16'] == '10-20 years') | (data['Q16'] == '20 or more years')]
# выбираем данные для России и для всех остальных стран
data_russia = data_10.query('Q4 == "Russia"')
data_other = data_10.query('Q4 != "Russia"')
# считаем количество респондентов в России и остальных странах
russia_count = len(data_russia)
other_count = len(data_other)
print('Количество респондентов, использующих методы машинного обучения более 10 лет (по странам):')
print(' Россия: {} чел.'.format(russia_count))
print('\nОстальные страны:')
other_counts = data_other['Q4'].value_counts().sort_values(ascending=False)
for country, count in other_counts.items():
    print('  {}: {} чел.'.format(country, count))

Количество респондентов, использующих методы машинного обучения более 10 лет (по странам):
 Россия: 1 чел.

Остальные страны:
  United States of America: 157 чел.
  India: 48 чел.
  Other: 41 чел.
  Brazil: 30 чел.
  France: 17 чел.
  United Kingdom of Great Britain and Northern Ireland: 16 чел.
  Spain: 14 чел.
  Japan: 12 чел.
  Portugal: 10 чел.
  Netherlands: 10 чел.
  Germany: 9 чел.
  Italy: 9 чел.
  Poland: 8 чел.
  Canada: 8 чел.
  Turkey: 8 чел.
  Australia: 8 чел.
  Mexico: 7 чел.
  Czech Republic: 6 чел.
  Belgium: 5 чел.
  South Korea: 5 чел.
  Singapore: 4 чел.
  Israel: 4 чел.
  Taiwan: 4 чел.
  Pakistan: 4 чел.
  China: 4 чел.
  Morocco: 3 чел.
  Romania: 3 чел.
  Indonesia: 3 чел.
  Egypt: 3 чел.
  Peru: 3 чел.
  Ireland: 2 чел.
  Nepal: 2 чел.
  United Arab Emirates: 2 чел.
  Thailand: 2 чел.
  South Africa: 2 чел.
  Colombia: 2 чел.
  Argentina: 2 чел.
  Chile: 2 чел.
  Bangladesh: 1 чел.
  Iran, Islamic Republic of...: 1 чел.
  Saudi Arabia: 1 чел.
  Sri Lanka: 1 чел